# Advent of Code 2019, Dyalog APL edition

To see a correct render of this notebook, check it out on [nbviewer](https://nbviewer.jupyter.org/github/xpqz/AoCDyalog/blob/master/Advent%20of%20Code%202019%20Dyalog%20APL.ipynb).

Annotated solutions in Dyalog APL. Why? A language that doesn't affect the way you think about programming is not worth knowing.

Note that part of the charm of AoC is that every user (or at least groups of users) gets their own unique data set. Some of the solutions below exploit quirks in my particular data set, and so may conceivably not work for the general case.

In [63]:
⍝ Helper functions and common settings
⎕FR ⎕PP ⎕IO←1287 34 0
assert←{⍺←'assertion failure' ⋄ 0∊⍵:⍺ ⎕signal 8 ⋄ shy←0}
lines←{⊃⎕NGET ⍵ 1}
line←{⊃lines ⍵}
sorted←{⍵[⍋⍵]}
sortbycol←⊢⌷⍨∘⊂∘⍒⌷⍤1
pairs←{↓(2÷⍨≢⍵) 2⍴⍵}
bin←{(32⍴2)⊤⍵}
dec←2∘⊥
and←{(bin ⍺)∧bin ⍵}
or←{(bin ⍺)∨bin ⍵}
rs←⊢∘-∘≢↑↓⍨∘-⍨
range←⊣+∘⍳-⍨
group←↓⊃¨,∘⊂⌸⊢/¨
replace←{⍺⍺(⍵⍵⌷⍨∘⊂⍳)@(∊∘⍺⍺)⍵}
cut←⊢⊂⍨≢⍤⊢⍴⊣↑≢

In [64]:
⍝ Some visualisation help, please
]box on -style=max -trains=tree -fns=on
]rows on

┌→─────────────────────────────────────┐
│Was ON -style=max -trains=tree -fns=on│
└──────────────────────────────────────┘

┌→─────┐
│Was ON│
└──────┘

### Day 1: The Tyranny of the Rocket Equation
https://adventofcode.com/2019/day/1

In [7]:
DAY01←⍎¨⊃⎕NGET'data/2019/01.txt'1
Fuel←{0⌈¯2+⌊⍵÷3}

In [9]:
⊢part1←+/Fuel DAY01
assert 3328306=part1

3328306

In [10]:
⊢part2←+/{⍵=0:0⋄(Fuel ⍵)+∇Fuel ⍵}¨DAY01
assert 4989588=part2

4989588

### Day 2: 1202 Program Alarm
https://adventofcode.com/2019/day/2

The first of MANY intcode tasks.

In [15]:
DAY02←⍎line'data/2019/02.txt'

In [22]:
]dinput
IntcodeV1←{
    ⍺←0                                          ⍝ Intcode interpreter. 
    (op p1 p2 p3)←4↑⍺↓⍵                          ⍝ Skip anything before ip (⍺) and take 4 cells
    op∊1 2:(⍺+4)∇((op-1)⌷⍵[p1](+,×)⍵[p2])@p3 ⊢ ⍵ ⍝ Addition and multiplication
    op=99:⍵[0]                                   ⍝ Exit
}

In [23]:
DAY02[1 2] ← 12 2
⊢part1←IntcodeV1 DAY02
assert 10566835=part1

10566835

In [24]:
Day02p2←{19690720=IntcodeV1 ((⊃1↑⍵)@1 2)⊢⍺:⊃1↑⍵⋄⍺∇1↓⍵}
(noun verb)←DAY02 Day02p2 {,⍳⍵ ⍵} 100 
⊢part2←verb+100×noun
assert 2347=part2

2347

### Day 3: Crossed Wires
https://adventofcode.com/2019/day/3

In [12]:
DAY03←1(↑,∘⍎↓)¨⎕CSV'data/2019/03.txt' ⍝ Convert each string to a letter and a number.
OFFSETS←4 2⍴0 1 0 ¯1 1 0 ¯1 0

In [13]:
]dinput
Follow←{
    ⍺←0 0
    0=≢⍵:⍺
    (dir steps)←⊃⍵
    seq←,⌿OFFSETS['UDRL'⍳dir;]∘.×1+⍳steps
    origin←(≢seq)⍴⊂¯2↑∊⍺
    ⍺,∊(seq+origin)∇1↓⍵                  
}

In [14]:
(path1 path2)←pairs∘Follow¨↓DAY03

In [15]:
crossings←1↓∪path1∩path2 ⍝ Find intersections; drop the start point which is shared.

In [16]:
⊢part1←⌊/(+/|)¨crossings
assert 860=part1

860

In [17]:
⊢part2←⌊/+/(path1⍳crossings),⍪path2⍳crossings
assert 9238=part2

9238

### Day 4: Secure Container
https://adventofcode.com/2019/day/4

In [57]:
DAY04←236491 713787

In [59]:
⊢part1←+/{enc←(6⍴10)⊤⍵⋄(∧/2≤/enc)∧∨/2=/enc}¨(0⊃DAY04)range 1+1⊃DAY04
assert 1169=part1

1169

In [60]:
ngr←{2∊≢¨((1,2≠/⊢)⍵)⊂⍵} ⍝ contains pair not part of larger group

In [61]:
⊢part2←+/{enc←(6⍴10)⊤⍵⋄(∧/2≤/enc)∧ngr enc}¨(0⊃DAY04)range 1+1⊃DAY04
assert 757=part2

757

### Day 5: Sunny with a Chance of Asteroids
https://adventofcode.com/2019/day/5

Better get used to it ⍨.

In [30]:
DAY05←⊢⌿⍎¨⎕CSV'data/2019/05.txt'

In [35]:
]dinput
IntcodeV2←{ ⍝ Intcode interpreter, mk2. Call as: 0 IntcodeV2 code
    state←⍵
    ev←{⍺=0: state[⍵] ⋄ ⍵}              ⍝ Position or immediate mode
    op p1 p2 p3←4↑⍺↓⍵                   ⍝ Skip anything before and take 4 cells
    params←4 4 2 2 3 3 4 4 1            ⍝ Number of parameters by opcode
    ops←(1+⍳8),99                       ⍝ Valid opcodes
    m3 m2 m1 o2 o1←(5⍴10)⊤op            ⍝ Unpack the param modes
    op←10⊥o2 o1                         ⍝ Repack the opcode, to go from (say) 1001 to 1
    count←params[ops⍳op]                ⍝ Number of params
    parmod←m1 m2 m3,⍪p1 p2 p3           ⍝ Table combining modes and params
    d1 d2 d3←3↑ev/(¯1+count)↑parmod     ⍝ Pick relevant number of params, and apply modes
    ip←⍺+count                          ⍝ Advance ip by the width of current instr
    op∊1 2:ip∇((op-1)⌷d1(+,×)d2)@p3⊢⍵   ⍝ Addition and multiplicatin
    op=3:ip∇INPUT@p1⊢⍵                  ⍝ Input
    op=4:d1,ip∇⍵                        ⍝ Output
    op∊5 6:⍵∇⍨ip d2⌷⍨(op-5)⌷d1(≠,=)0    ⍝ Jumps 
    op∊7 8:ip∇((op-7)⌷d1(<,=)d2)@p3⊢⍵   ⍝ Comparison < or =
    op=99:⍬                             ⍝ Exit
}

In [36]:
INPUT←1
⊢part1←⊃¯1↑0 IntcodeV2 DAY05
assert 16209841=part1

16209841

In [39]:
INPUT← 5
⊢part2←⊃0 IntcodeV2 DAY05
assert 8834787=part2

8834787

### Day 6: Universal Orbit Map
https://adventofcode.com/2019/day/6

In [53]:
DAY06←↓⍉↑'\w+'⎕S'&'¨lines'data/2019/06.txt'
PARENTS←⊃⍳/⊖DAY06
Path←{3::⍵⋄⍵,∇⍵⊃PARENTS}

In [55]:
⊢part1←≢∊Path¨PARENTS
assert part1=292387

292387

In [63]:
⊢part2←{¯2+≢⍺(∪~∩)⍵}/Path¨(1⊃DAY06)⍳'YOU' 'SAN'
assert 433=part2

433

### Day 7: Amplification Circuit
https://adventofcode.com/2019/day/7

Already fed up.

This is a resumable Intcode iterpreter -- fed a state vector (code ip rb input output) it will execute until it's either terminated normally on opcode 99, or blocked on input.

Yes, it's a tradfn. Don't @ me. I wrote it so you won't have to, and we can all get back to our lives.

This question's instructions are somewhat misunderstandable. The suggestion is that there is some difference in the 'wiring' between parts 1 and 2; this is either not the case, or part 1 works equally well with the circular wiring suggested in part 2.

Also, part 2 states:

    Don't restart the Amplifier Controller Software on any amplifier during this process. 
    Each one should continue receiving and sending signals until it halts.
    All signals sent or received in this process will be between pairs of amplifiers except 
    the very first signal and the very last signal. To start the process, a 0 signal is sent 
    to amplifier A's input exactly once.
    
This actually means "exactly once (per phase setting)", rather than actually exactly once. So in practice, both parts are identical apart from the phases to try.

In [3]:
'pmat'⎕CY'dfns'
DAY07←⊢⌿⍎¨⎕CSV'data/2019/07.txt'

In [4]:
]dinput
r←IntcodeV3 argv;code;ip;rb;in;out;halt;ev;op;p1;p2;p3;params;ops;m3;m2;m1;o2;o1;count;parmod;d1;d2;d3
(code ip rb in out halt)←argv
:If halt
    r←argv
    :Return
:EndIf
ev←{⍺=0:⍵⊃code⋄⍵}                          ⍝ Position or immediate mode
:While 1
    (op p1 p2 p3)←4↑ip↓code                ⍝ Skip anything before and take 4 cells
    :If op=99
        r←code ip rb in out 1
        :Return
    :EndIf
                      
    params←4 4 2 2 3 3 4 4 1              ⍝ Number of parameters by opcode
    ops←99,⍨1+⍳8                          ⍝ Valid opcodes
    (m3 m2 m1 o2 o1)←op⊤⍨5⍴10             ⍝ Unpack the param modes
    op←10⊥o2 o1                           ⍝ Repack the opcode, to go from (say) 1001 to 1
    count←params[ops⍳op]                  ⍝ Number of params
    parmod←m1 m2 m3,⍪p1 p2 p3             ⍝ Table combining modes and params
    (d1 d2 d3)←3↑ev/(¯1+count)↑parmod     ⍝ Pick relevant number of params, and apply modes

    ip+←count                             ⍝ Advance ip by the width of current instr
    :If op∊1 2                            ⍝ Addition or multiplication
        code[p3]←(op-1)⌷d1(+,×)d2
    :ElseIf op=3                          ⍝ Input
        :If 0=≢in                         ⍝ Blocked on input
            r←code (ip-count) rb in out 0 ⍝ Reverse one instruction
            :Return
        :EndIf
        code[p1]←0⊃in
        in←1↓in
    :ElseIf op=4                          ⍝ Output
        out,←d1
    :ElseIf op∊5 6                        ⍝ Jumps
        ip←ip d2⌷⍨(op-5)⌷d1(≠,=)0
    :ElseIf op∊7 8                        ⍝ Comparisons
        code[p3]←(op-7)⌷d1(<,=)d2
    :Else
        assert 0 ⍝ Unknown opcode
    :End
:EndWhile

In [11]:
]dinput
RunPhase←{
    amps←⍵
    amps[;3],←⍺⋄amps←↑IntcodeV3¨↓amps
    amps[0;3],←0
    amps[0;]←IntcodeV3 0⌷amps

    RunAmps←{
        ampId←5|⍵+1
        amps[ampId;3],←0⊃⊖⊃amps[⍵;4]
        ⊢amps[ampId;]←IntcodeV3 ampId⌷amps
    }

    _←{RunAmps¨⍳5}⍣{amps[4;5]}⊢⍬
    amps
}

In [12]:
]dinput
Day07←{
    state←(↑5⍴⊂(⍺)0 0(,⍬)(,⍬)0)
    ⌈/{⊃¯1↑⊃(4 4)⌷⍵ RunPhase state}¨⍵
}

In [14]:
⊢part1←DAY07 Day07 ↓pmat 5
assert 262086=part1

262086

In [15]:
⊢part2←DAY07 Day07 {5 6 7 8 9[⍵]}¨↓pmat 5
assert 5371621=part2

5371621

### Day 8: Space Image Format
https://adventofcode.com/2019/day/8

Briefly back to non-intcode, and proper arrays!

In [192]:
DAY08←100 6 25⍴⍎¨line'data/2019/08.txt'

In [193]:
Z←⊃⍋{+/0=∊DAY08[⍵;;]}¨⍳100 ⍝ Pick layer with the fewest zeros

In [194]:
⊢part1←(+/1=∊DAY08[Z;;])×+/2=∊DAY08[Z;;]
assert 1935=part1

1935

Part 2: decode the image, with the value 2 being transparent. We can use the dyadic form of transpose to great effect here, to group the 100 values that make up each pixel into vectors.

In [201]:
img←{(⊃⍸2≠⍵)⊃⍵}¨↓2 0 1⍉DAY08 ⍝ Find first non-2 value at each point along the 100 layers
' *'[img]                    ⍝ Image reads CFLUL

┌→────────────────────────┐
↓ ** **** * * * * │
│* * * * * * * │
│* *** * * * * │
│* * * * * * │
│* * * * * * * │
│ ** * **** ** **** │
└─────────────────────────┘

### Day 9: Sensor Boost
https://adventofcode.com/2019/day/9

Yeah. The fun didn't last long, did it. Our new Intcode interpreter needs to support the 'relative base' operation, opcode 9, and also allow addressing an arbitrary distance beyond the given program. The intention here is that this is the final version of Intcode interpreter. 

I learnt a lot from [voidhawk42](https://github.com/voidhawk42/aoc2019apl/blob/master/p09.dyalog) on this.

In [307]:
DAY09←⊢⌿⍎¨⎕CSV'data/2019/09.txt'

Given the state of a running intcode program, exectute one instruction and return the new state.

In [53]:
]dinput
Step←{ 
    (code ip rb in out halt blocked)←⍵
    evalPar←{⍺=0:⍵⊃code⋄⍺=2:(⍵+rb)⊃code⋄⍵}
    applyRelBase←{⍺=2:⍵+rb⋄⍵}
    (op p1 p2 p3)←4↑ip↓code
    params←4 4 2 2 3 3 4 4 2 1
    ops←99,⍨1+⍳9
    (m3 m2 m1 o2 o1)←op⊤⍨5⍴10
    op←10⊥o2 o1
    count←params[ops⍳op]
    parmod←(¯1+count)↑m1 m2 m3,⍪p1 p2 p3
    read←3↑evalPar/parmod
    write←3↑applyRelBase/parmod
    ip+←count
    op∊1 2:code ip rb in out 0 0⊣code[2⊃write]←(op-1)⌷read[0](+,×)1⊃read
    (op=3)∧0=≢in:code (ip-count) rb in out 0 1
    op=3:code ip rb (1↓in) out 0 0⊣code[0⊃write]←0⊃in
    op=4:code ip rb in (out,0⊃read) 0 0
    op∊5 6:code (ip (1⊃read)⌷⍨(op-5)⌷read[0](≠,=)0) rb in out 0 0
    op∊7 8:code ip rb in out 0 0⊣code[2⊃write]←(op-7)⌷read[0](<,=)read[1]
    op=9:code ip (rb+0⊃read) in out 0 0
    op=99:code ip rb in out 1 0
    assert 0
}

Now we can run a program to completion using power ⍣. Note that we need to pre-allocate sufficient 'RAM' for the execution.

In [309]:
Intcode←{⍺←⍬⋄Step⍣{5⊃⍺}⊢(⍵,512⍴0)0 0(,⍺)(,⍬)0 0} ⍝ ⍺ is input vector, ⍵ is the program

In [310]:
⊢part1←⊃4⊃1 Intcode DAY09
assert 2171728567=part1

2171728567

In [311]:
⊢part2←⊃4⊃2 Intcode DAY09
assert 49815=part2

49815

### Day 10: Monitoring Station
https://adventofcode.com/2019/day/10

Trigonometry? For each asteroid, calculate angle to every other asteroid, and find the max count of unique angles.

In [182]:
DAY10←↑lines'data/2019/10.txt'
ASTR←⍸'#'=DAY10

In [183]:
Deg←{0=⍵:0⋄(180÷○∘÷)⍵}
ATan2←12○⊣+0J1×⊢
Angle←{v←360-360|180+Deg ATan2/⍵-⍺⋄v=360:0⋄v} ⍝ 0 and 360 pointing 'north' and increasing clockwise

In [184]:
GROUPS←(≢∪)¨↓ASTR∘.Angle ASTR
WINNER←⊃⍒GROUPS
⊢part1←WINNER⊃GROUPS
assert 334=part1

334

Part 2: complete vaporization by giant laser. Starting pointing up, eliminate the first asteroid visible, and repeat the process whilst rotating clockwise. Which is the 200th asteroid to be eliminated?

For this we need both angles and distances.

In [185]:
Dist←{.5*⍨+/2*⍨⍵-⍺}
Visibility←{(⍺ Angle ⍵)((⍺ Dist ⍵),⍵)}

In [186]:
VIZ←group sorted (⊃ASTR[WINNER])∘Visibility¨ASTR

The VIZ vector now holds groups consisting of angle and triplets of distance and coordinates, in ascending order. We need to find the 200th coordinate, by iterating over the angles, clockwise, picking the first non-seen item from each distance-ordered list. If we remix the coordinate parts of VIZ, we should get them in the order required, and then we can pick directly the coordinate at index 199.

In [187]:
ZAPPED←⊃0⌷↓⍉↑{{1↓⊃⍵}¨1⊃⍵}¨VIZ ⍝ Pick out only the coordinates, and remix/zip ↓⍉↑.

In [188]:
⊢part2←199⊃ZAPPED
assert part2≡19 11 ⍝ Submittable result is 1119, y+x×100

┌→────┐
│19 11│
└~────┘

### Day 11: Space Police
https://adventofcode.com/2019/day/11

And with that fun out of the way, normal Intcode service returns.

In [312]:
DAY11←⊢⌿⍎¨⎕CSV'data/2019/11.txt'
Turn←{⍺ ⍵⌷4 2⍴3 1 0 2 1 3 2 0}
Move←{⊃⍵⌷⍺+(¯1 0)(0 1)(1 0)(0 ¯1)}

In [313]:
]dinput
Day11←{
    ⍺←0 
    code←⍵
    panel←100 100⍴0
    panel[50;50]←⍺
    robot←{⍵⊣⍵.(state dir count pos)←((code,512⍴0)0 0(,⍬)(,⍬)0 0) 0 0 (50 50)}⎕NS''
    painted←100 100⍴0
    _←{ 
        robot.state[3]←⊂,robot.pos⌷panel             ⍝ Input: current tile colour
        robot.state[4]←⊂,⍬                           ⍝ Reset output buffer
        robot.state←Step⍣{(5⊃⍺)∨2=≢4⊃⍺}⊢robot.state  ⍝ Run robot until two outputs or halt
        5⊃robot.state:⍬                              ⍝ Halted
        (colour dir)←4⊃robot.state
        panel[⊂robot.pos]←colour                     ⍝ Paint our current tile
        painted[⊂robot.pos]+←colour=1
        robot.dir←robot.dir Turn dir                 ⍝ Turn left or right, as directed
        ⊢robot.pos←(⊂robot.pos) Move robot.dir       ⍝ Take one step in new dir
    }⍣{5⊃robot.state}⊢⍬
    (painted) (panel)
}

In [314]:
(painted _)←Day11 DAY11
⊢part1←+/0≠∊painted
assert 2088=part1

2088

In [315]:
(_ panel)←1 Day11 DAY11
(' *'[panel])[50+⍳6;51+⍳39] ⍝ Part 2: message reads URCAFLCP

┌→──────────────────────────────────────┐
↓* * *** ** ** **** * ** *** │
│* * * * * * * * * * * * * *│
│* * * * * * * *** * * * *│
│* * *** * **** * * * *** │
│* * * * * * * * * * * * * │
│ ** * * ** * * * **** ** * │
└───────────────────────────────────────┘

### Day 12: The N-Body Problem
https://adventofcode.com/2019/day/12

In [278]:
DAY12←(4 1 ¯15)(¯8 ¯10 1)(9 4 ¯5)(¯2 6 4) ⍝ Flipped from xyz to zyx format

In [279]:
Gravity←{+/⍵∘.(<->)⍵} ⍝ Return velocity change, a.k.a acceleration
Energy←×⍥(+/∘|)
ApplyForces←{(p v)←⍵⋄(p+nv)(nv←v+Gravity p)}

In [280]:
⊢part1←+/Energy⌿↑ApplyForces⍣1000⊢DAY12 ((0 0 0)(0 0 0)(0 0 0)(0 0 0))
assert 8625=part1

8625

Nice.

Part 2 asks us to determine the number of steps that must occur before all of the moons' positions and velocities exactly match a previous point in time. There are strong hints that the number of iterations will likely be too large for us to brute. 

This is a trickier problem.

The key insight to exploit is that the column vectors for x, y and z are independent and periodic (or there would be no solution), so we search for the common period.

In [281]:
]dinput
Periods←{
    startState←↓⍉↑⍵                ⍝ zip to make col vectors
    periods←¯1 ¯1 ¯1               ⍝ zyx
    count←2
    _←{
        periods[⍸(¯1=periods)∧(↓⍉↑⊃0⌷⍵)≡⍤0⊢startState]←count
        ApplyForces ⍵
    }⍣{count+←1⋄~¯1∊periods}⊢ApplyForces ⍵ ((0 0 0)(0 0 0)(0 0 0)(0 0 0))
    periods
}

In [282]:
⊢periods←Periods DAY12

┌→──────────────────┐
│193052 96236 286332│
└~──────────────────┘

So that gives us the per-component periods, but what is the common period they must share? Fortunately for us, APL has lowest common multiple built in, which is handy.

In [285]:
⊢part2←∧/periods ⍝ wow apl :)
assert 332477126821644=part2

332477126821644

### Day 13: Care Package
https://adventofcode.com/2019/day/13

For that we now have to pay the mandated Intcode tax.

Part 1: run the program, count the number of output groups of three ending in a 2.

In [316]:
DAY13←⊢⌿⍎¨⎕CSV'data/2019/13.txt'

In [317]:
⊢part1←+/2=2⊃¨↓(3÷⍨≢out) 3⍴out←4⊃Step⍣{5⊃⍺}⊢(DAY13,512⍴0)0 0(,⍬)(,⍬)0 0
assert 193=part1

193

In [367]:
]dinput
Breakout←{
    code←⍵
    code[0]←2 ⍝ Enable unlimited quarters mode
    final←{
        state←Step⍣{(6⊃⍺)∨5⊃⍺}⊢⍵                       ⍝ Execute until blocked or halted
        5⊃state:state                                  ⍝ Game over
        buf←↑3 cut 4⊃state                             ⍝ Matrix from triplets
        data←⊢/buf                                  
        state[3],←buf[data⍳3;0](<->)buf[⊃¯1↑⍸4⍷data;0] ⍝ Paddle tracks last ball in x
        state[4]←⊂,⍬                                   ⍝ Reset output
        state
    }⍣{5⊃⍺}⊢(code,512⍴0)0 0(,0)(,⍬)0 0
    result←↑3 cut 4⊃final
    scoreidx←result[;0 1]⍳¯1 0
    result[scoreidx;2]
}

In [368]:
⊢part2←Breakout DAY13
assert 10547=part2

10547

### Day 14: Space Stoichiometry
https://adventofcode.com/2019/day/14

Solution as presented here heavily influenced by [voidhawk42](https://github.com/voidhawk42/aoc2019apl/blob/master/p14.dyalog). Python [version](https://github.com/xpqz/aoc-19/blob/master/day14.py).

In [481]:
'segs'⎕CY'dfns'
DAY14←↑{↓⍉↑⍵}¨{2 cut {6::⍵⋄⍎⍵}¨', =>' segs ⍵}¨lines'data/2019/14.txt'

In [482]:
TARGETS←(⊂'ORE'),⍨⊃,/¯1↑¨DAY14[;1]

Now we build a table of all the formulae, showing target amounts as positive values along the diagonal, and the ingredient 'costs' as negative weights.

In other words, if we have the rule

    (⊂1 1 2) ('HVXJL' 'JHGQ' 'ZQFQ')

we can expect the entry in the HVXJL and JHGQ cols to be ¯1 and that for the target ZQFQ to be 2.

Dyalog's [selective assigment](http://help.dyalog.com/18.0/#Language/Primitive%20Functions/Assignment%20Selective.htm).

In [484]:
FORMULAE←-↑{⍺@(TARGETS⍳⍵)⊢0⍴⍨≢TARGETS}/DAY14
(0 0⍉FORMULAE)×←¯1 ⍝ Flip sign along diagonal via selective assignment

In [490]:
]dinput
React←{ ⍝ React amnt
    ⊃-¯1↑{
        ∧/0≤¯1↓⍵:⍵                           ⍝ We're done. ORE is the last item.
        element←⊃⍸0>⍵                        ⍝ Pick first unfulfilled (negative) element
        items←element⌷FORMULAE               ⍝ ...and its corresponding ingredient weights
        ∇⍵+items×⌈(-element⌷⍵)÷element⌷items ⍝ Adjust remaining weights and add to queue
    } ⍵×FORMULAE⌷⍨TARGETS⍳⊂'FUEL'
}

In [489]:
⊢part1←React 1
assert 216477=part1

216477

Binary search

In [497]:
⊢part2←1 {⍺=⍵:⍵ ⋄ 1e12>React⊢m←⌈2÷⍨⍺+⍵:m∇⍵-1 ⋄ ⍺∇m} 2×⍣{1e12<React ⍺}⊢1
assert 11788286=part2

11788286

### Day 15: Oxygen System
https://adventofcode.com/2019/day/15

Guess what. Intcode AND graphy path findingy things.

We start with a breadth-first traversal of the maze to map it out. We can then apply Dijkstra's algorithm for parts 1 and 2. We could have tapped out the answer for part 1 directly from the BFS at the cost of some complexity.

In [49]:
DAY15←⊢⌿⍎¨⎕CSV'data/2019/15.txt'
ICState←{(⍵,512⍴0)0 0(,⍬)(,⍬)0 0}
Move←{ic←⍺ ⋄ ic[3],←⍵ ⋄ Step⍣{(6⊃⍺)∨5⊃⍺}⊢ic}

In [50]:
]dinput
N4←{
    (cur state)←⍵
    pos←(⊂cur)(-,+)(0 1)(1 0)                 ⍝ W N E S = 3 1 4 2
    unvisited←(≢⍺)=⍺⍳pos                         
    new←state∘Move¨3 1 4 2                    ⍝ Apply each move command (classify location)
    tiles←∊¯1↑¨(↑new)[;4]                     ⍝ Last item of each output buffer
    valid←unvisited∧0≠tiles                   ⍝ Unvisited non-wall positions
    (↓⍉↑(valid/pos)(valid/new))(pos/⍨2=tiles)
}

In [51]:
]dinput
YABFS←{ ⍝ Yet Another Breadth First Search
    spout←⍬
    ⍬ {
        0=≢⍵:⍺ spout
        head←0⊃⍵
        (newStates cand)←⍺ N4 head
        _←{0≠≢⍵:⊢spout⊢←⍵⋄⍬} cand
        (⍺,¯1↓head)∇(1↓⍵),newStates
    } ,⊂(100 100) (ICState ⍵) ⍝ Y X state-vec
}

In [54]:
(empties spout)←YABFS DAY15

Now we have a graph, and a target location, which should mean we're done with the Intcode bits, thankfully.

Looking back through the AoC archives, we've used Dijkstra's several times that we can use as a basis. This time, however -- and we should have done this ages ago -- let's have a dictionary. Come on, Dyalog, you know it makes sense!

In [73]:
]LINK.Create heapq src/heapq
]LINK.Create dict src/dict

┌→───────────────────────────────────────────────────────────────────────────┐
│Already linked: #.dict.heapq → /Users/stefan/work/dyalog/AoCDyalog/src/heapq│
└────────────────────────────────────────────────────────────────────────────┘

┌→─────────────────────────────────────────────────────────────────────────┐
│Already linked: #.dict.dict → /Users/stefan/work/dyalog/AoCDyalog/src/dict│
└──────────────────────────────────────────────────────────────────────────┘

In [82]:
Neighbours←{⊂¨(⍸×⍺)∩⍵+(¯1 0)(1 0)(0 ¯1)(0 1)}
UnwindPath←{p←⍺⋄(s e)←⍵⋄⍬{⍵≡s:⊖⍺⋄(⍺,⍵)∇p ##.dict.Get ⍵}e}

In [100]:
]dinput
Dijkstra←{
    ⍝ Dijkstra's without cut-off
    ⍝ See https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm
    cost←⍵ ##.dict.Create 0 ⋄ cost.Default←⌊/⍬
    cameFrom←⍵ ##.dict.Create ⍵
    Neigh←⍺⍺
    Pick←{oldCost←cost ##.dict.Get ⍵⋄⍺<oldCost}                
    _←{
        (queue queueItem)←##.heapq.Pop ⍵
        current←1⊃queueItem
        newCost←1+cost ##.dict.Get current
        valid←(newCost∘Pick¨n)/n←Neigh current             ⍝ New, or better?
        _←cameFrom ##.dict.Set (valid) ((≢valid)⍴⊂current) ⍝ Update path dictionary
        _←cost ##.dict.Set (valid) ((≢valid)⍴newCost)      ⍝ Any changed costs
        queue ##.heapq.Push ↓⍉↑((≢valid)⍴newCost)(valid)   ⍝ Enqueue connected nodes
    }⍣{##.heapq.Empty ⍺} ##.heapq.Push ⊂0 ⍵
    cameFrom
}

In [101]:
graph←120 120⍴0
graph[empties]←1

In [102]:
cf←(graph∘Neighbours Dijkstra) ⊂100 100

In [103]:
⊢part1←≢cf UnwindPath (⊂100 100)(⊂88 80)
assert 248=part1

248

Part 2 can be expressed as finding the longest shortest path between the spout and any other open space. 

In [87]:
cf←(graph∘Neighbours Dijkstra) ⊂88 80

In [88]:
⊢part2←⌈/{≢cf UnwindPath (⊂⍺)(⊂⍵)}⌿↑((≢empties)⍴⊂88 80)(empties)
assert 382=part2

382

In retrospect, we could have used the Dijkstra routine directly, in place of the 'YABFS' function above, by having it record the location of the spout. But that would have made the Dijkstra rather fugly.

### Day 16: Flawed Frequency Transmission
https://adventofcode.com/2019/day/16

Ok, we're back on APL home-court advantage. It starts easy enough...

In [114]:
⎕io←1
DAY16←⍎¨line'data/2019/16.txt'

In [115]:
Coeff←{⍺↑1↓∊(1+⌈⍺÷4×⍵)⍴⊂∊⍵∘{⍺⍴⍵}¨0 1 0 ¯1}
Phase←{10||(↑(≢⍵)Coeff¨⍳≢⍵)+.×⍵}

In [116]:
⊢part1←8↑Phase⍣100⊢DAY16
assert 5 8 1 0 0 1 0 5≡part1

┌→──────────────┐
│5 8 1 0 0 1 0 5│
└~──────────────┘

In part 2, we're asked to repeat our input signal 10,000 times, making the naive approach intractable. However, as the coefficient matrix is upper-diagonal, we can skip a whole lot. Look at the coefficients for the example given:

In [117]:
{↑(≢⍵)Coeff¨⍳≢⍵}⍳8

┌→─────────────────┐
↓1 0 ¯1 0 1 0 ¯1 0│
│0 1 1 0 0 ¯1 ¯1 0│
│0 0 1 1 1 0 0 0│
│0 0 0 1 1 1 1 0│
│0 0 0 0 1 1 1 1│
│0 0 0 0 0 1 1 1│
│0 0 0 0 0 0 1 1│
│0 0 0 0 0 0 0 1│
└~─────────────────┘

The first 7 digits of our input now represents the offset to the sub-sequence we're interested in. Given the zeroes in the coefficient matrix, only the sub-sequence actually contributes, so can skip everything before. Each digit depends on the previous (digit n depends on digit n-1), as the zeros are followed by ones.

In [139]:
⊢offset←10⊥7↑DAY16

5970443

In [141]:
⊢part2←8↑{1↓⊖10|+\(⊖⍵),0}⍣100⊢offset↓∊10000⍴⊂DAY16
assert 4 1 7 8 1 2 8 7≡part2

┌→──────────────┐
│4 1 7 8 1 2 8 7│
└~──────────────┘

That was nice, wasn't it? I learnt something after trying first to do:

    {1↓⊖{10|⍺+⍵}\(⊖⍵),0}
    
which is crushingly slow. Scan with a dfn operand ends up being O(n^2), where as +\ hits a highly efficient optimised idiom. 

### Day 17: Set and Forget
https://adventofcode.com/2019/day/17

Anyway. Let's put that lovely problem aside, and get back to some Intcode :/

In [157]:
⎕IO←0
DAY17←⊢⌿⍎¨⎕CSV'data/2019/17.txt'

In [158]:
]dinput
MakeGraph←{
    graph←↑10(≠⊆⊢)4⊃Step⍣{(6⊃⍺)∨5⊃⍺}⊢(⍵,10000⍴0)0 0(,⍬)(,⍬)0 0
    heading←⊃(∪∊graph)~46 35
    (1@(⍸35=graph)⊢(⍴graph)⍴0)(⊃⍸heading=graph)heading
}

In [159]:
(g loc hd)←MakeGraph DAY17

In [160]:
cross←3 3 ⍴ 0 1 0 1 1 1 0 1 0   ⍝ All intersections look like a 3x3 'cross'
⊢part1←+/×/↑⍸({cross≡⍵}⌺3 3) g
assert 8408=part1

8408